<a href="https://colab.research.google.com/github/katarinagresova/benchmarks/blob/main/ecoli_promoters/E_coli_Promoter_Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# E. coli Promoter Data Preparation

This notebook is based in E. coli Data Processing notebook from [Genomic ULMFiT](https://github.com/kheyer/Genomic-ULMFiT). It creates he following dataset:

#### E. coli Promoter Classification
This dataframe will contain E. coli promoters, defined as -100/+50 base pairs from each annotated transcription start site. The classification data will also include negative examples of the same length taken from regions not including a TSS. Taking promoter sequences based on TSS locations runs the risk of putting highly similar promoters into the dataset. To construct meaningful test and validation sets to evaluate the model, we must ensure that sequences in the test/validation sets are sufficiently different from sequences in the training set. This is done using a method described below.

# Data Source
E. coli genomic data and TSS locations are taken from the E. coli reference genome genbank file available at [NCBI](https://www.ncbi.nlm.nih.gov/genome/?term=escherichia%20coli)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
!pip3 install biopython

     |████████████████████████████████| 2.3MB 4.0MB/s 


In [3]:
from fastai import *
from fastai.text import *
from Bio import Seq
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import FeatureLocation, CompoundLocation
import networkx as nx
import urllib
import os

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Choose path where data will be stored.

In [5]:
path = Path('/content/drive/My Drive/Benchmarks/ecoli_promoters/')

In [6]:
fname = 'GCF_000005845.2_ASM584v2_genomic.gbff'
gz_path_to_file = Path(path, fname + ".gz")

In [7]:
u = "https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gbff.gz"
urllib.request.urlretrieve(u, gz_path_to_file)

(PosixPath('/content/drive/My Drive/Benchmarks/ecoli_promoters/GCF_000005845.2_ASM584v2_genomic.gbff.gz'),
 <http.client.HTTPMessage at 0x7ff44525da90>)

Unpack donwloaded file from .gz to be able to use it later.

In [8]:
import gzip
import shutil
with gzip.open(gz_path_to_file, 'rb') as f_in:
    with open(path/fname, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# Classification Data

In [9]:
GB = next(SeqIO.parse(path/fname, "genbank"))

In [10]:
# Extract promoter regions
# All sequences saved as sense strands
def extract_promoter(feature):
    quals = list(feature.qualifiers.keys())
    
    if 'gene' in quals:
        gene = feature.qualifiers['gene']
    else:
        gene = None
        
    if 'locus_tag' in quals:
        locus = feature.qualifiers['locus_tag']
    else:
        locus = None
        
    cds_loc = str(feature.location)
    cds_start = feature.location.start.position
    cds_end = feature.location.end.position
    p_start = cds_start - 100
    p_end = cds_end + 100
    
    if feature.strand == -1:
        orient = 'reverse'
        promoter = GB[cds_end-50:p_end].reverse_complement().seq.__str__()
        promoter_loc = f"[{cds_end-50}:{p_end}]" + f"{cds_loc[-3:]}"
                
    else:
        orient = 'forward'
        promoter = GB[p_start:cds_start+50].seq.__str__()
        promoter_loc = f"[{p_start}:{cds_start+50}]" + f"{cds_loc[-3:]}"
                    
    promoter_data = [gene, locus, cds_loc, promoter_loc, orient, promoter, 1]
    
    return promoter_data

In [11]:
# Extract negative examples
def extract_gene(feature):
    quals = list(feature.qualifiers.keys())
    
    if 'gene' in quals:
        gene = feature.qualifiers['gene']
    else:
        gene = None
        
    if 'locus_tag' in quals:
        locus = feature.qualifiers['locus_tag']
    else:
        locus = None
        
    cds_loc = str(feature.location)
    cds_start = feature.location.start.position
    cds_end = feature.location.end.position
    
    gene_len = (cds_end-50) - (cds_start + 50)
    
    if gene_len > 150:
        rand_start = np.random.randint(cds_start+50, cds_end-200)
        rand_gene = GB[rand_start:rand_start+150]
        rand_gene_loc = f"[{rand_start}:{rand_start+150}]" + f"{cds_loc[-3:]}"
            
        if feature.strand == -1:
            orient = 'reverse'
            rand_gene = rand_gene.reverse_complement().seq.__str__()
            
        else:
            orient = 'forward'
            rand_gene = rand_gene.seq.__str__()
            
        gene_data = [gene, locus, cds_loc, rand_gene_loc, orient, rand_gene, 0]
        return gene_data

    else:
        return False

In [12]:
proms = []
for feature in GB.features:
    if feature.type == 'CDS':
        proms.append(extract_promoter(feature))

In [13]:
prom_df = pd.DataFrame(proms, columns=['Gene', 'Locus', 'Location', 'Sample Location', 'Orientation', 'Sequence', 'Promoter'])

In [14]:
genes = []
for feature in GB.features:
    if feature.type == 'CDS':
        gene = extract_gene(feature)
        if gene:
            genes.append(gene)

In [15]:
gene_df = pd.DataFrame(genes, columns=['Gene', 'Locus', 'Location', 'Sample Location', 'Orientation', 'Sequence', 'Promoter'])

In [16]:
# Approximately 4000 positive and negative examples
prom_df.shape, gene_df.shape

((4357, 7), (4014, 7))

In [17]:
promoter_data = pd.concat([prom_df, gene_df])

In [18]:
promoter_data.shape

(8371, 7)

In [19]:
promoter_data.head()

,Gene,Locus,Location,Sample Location,Orientation,Sequence,Promoter
0,[thrL],[b0001],[189:255](+),[89:239](+),forward,CGTGAGTAAATTAAAATTTTATTGACTTAGGTCACTAAATACTTTA...,1
1,[thrA],[b0002],[336:2799](+),[236:386](+),forward,AGGTAACGGTGCGGGCTGACGCGTACAGGAAACACAGAAAAAAGCC...,1
2,[thrB],[b0003],[2800:3733](+),[2700:2850](+),forward,CCGTTGGTACTGCGCGGATATGGTGCGGGCAATGACGTTACAGCTG...,1
3,[thrC],[b0004],[3733:5020](+),[3633:3783](+),forward,CGTTGCCGACTGGTTGGGTAAGAACTACCTGCAAAATCAGGAAGGT...,1
4,[yaaX],[b0005],[5233:5530](+),[5133:5283](+),forward,GTAACTTAGAGATTAGGATTGCGGAGAATAACAACCGCCGTTCTCA...,1


# Sequence Similarity

It's highly likely that there are promoter sequences in the dataset that share sequence similarity. We need to make sure similar sequences don't get split into training and validation/test sets.

We use a BLAST search to identify homologies between sequences. We use the information from the BLAST search to create a graph, where each sequence is a node connected to other sequences that share homology. We then grab a maximal independent set from the graph. We can safely put sequences identified as independent into validation/test sets.

Sequences sharing more than 15 base pairs of homology are considered similar.

10% of sequences will be set aside for the test set. Of the 90% of sequences that remain, 10% of those will be set aside for a validation set. The remaining sequences will be used for training.

In [20]:
df = promoter_data

In [21]:
# Write sequences to fasta file
with open('sequences.fasta', 'w') as out:
    for i in range(len(df)):
        out.write('>' + str(df.index[i]) + '\n' + df.Sequence.iloc[i] + '\n')

In [22]:
!apt-get install ncbi-blast+

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  ncbi-data
The following NEW packages will be installed:
  ncbi-blast+ ncbi-data
0 upgraded, 2 newly installed, 0 to remove and 16 not upgraded.
Need to get 13.1 MB of archives.
After this operation, 66.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 ncbi-data all 6.1.20170106-2 [3,645 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 ncbi-blast+ amd64 2.6.0-1 [9,446 kB]
Fetched 13.1 MB in 3s (3,871 kB/s)
Selecting previously unselected package ncbi-data.
(Reading database ... 145483 files and directories currently installed.)
Preparing to unpack .../ncbi-data_6.1.20170106-2_all.deb ...
Unpacking ncbi-data (6.1.20170106-2) ...
Selecting previously unselected package ncbi-blast+.
Preparing to unpack .../ncbi-blast+_2.6.0-1_amd64.deb ...
Unpacking ncbi-blast+ (2.6.0-1) 

In [23]:
!makeblastdb -in sequences.fasta -dbtype 'nucl' -out hom_arm_db



Building a new DB, current time: 01/13/2021 07:03:26
New DB name:   /content/hom_arm_db
New DB title:  sequences.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 8371 sequences in 0.159079 seconds.


In [24]:
!blastn -db hom_arm_db -query sequences.fasta -word_size 15 -out out_1.csv -outfmt 6

In [25]:
blast_cols = ['Query', 'Subject', '%ident', 'Length', 'Mismatch', 'Gaps', 'qstart', 'qend',
                 'sstart', 'send', 'evalue', 'bitscore']
df_out = pd.read_csv('out_1.csv', sep='\t', header=None)
df_out.columns = blast_cols
df_out = df_out[df_out.Query != df_out.Subject]
df_out.reset_index(inplace=True, drop=True)

In [26]:
df_inter = df_out[['Query', 'Subject']]
df_inter = df_inter.drop_duplicates()

In [27]:
def link_list(hom, overlaps, hom_names):
    links = overlaps[overlaps.Query == hom].Subject.values
    links = links[np.isin(links, hom_names)]
    links = [(hom, i) for i in links]

    return links

In [28]:
def max_pooling(hom_names_list, hom_overlap_list, link_dict_in):
    G = nx.Graph()
    G.add_nodes_from(hom_names_list)

    for i in hom_overlap_list:
        G.add_edges_from(link_dict_in[i])

    return nx.maximal_independent_set(G)

In [29]:
link_dict = {}
for hom in df_inter.Query.unique():
    link_dict[hom] = link_list(hom, df_inter, df.index.values)

In [30]:
ind_set = np.array(max_pooling(df.index.values, df_inter.Query.unique(), link_dict))

7907 sequences are independent, as defined as having less than 15 base pairs of homology to another sequence in the dataset

In [31]:
len(ind_set)

2638

In [32]:
df['Independent'] = df.index.map(lambda x: x in ind_set)

In [33]:
df.head(10)

,Gene,Locus,Location,Sample Location,Orientation,Sequence,Promoter,Independent
0,[thrL],[b0001],[189:255](+),[89:239](+),forward,CGTGAGTAAATTAAAATTTTATTGACTTAGGTCACTAAATACTTTA...,1,True
1,[thrA],[b0002],[336:2799](+),[236:386](+),forward,AGGTAACGGTGCGGGCTGACGCGTACAGGAAACACAGAAAAAAGCC...,1,True
2,[thrB],[b0003],[2800:3733](+),[2700:2850](+),forward,CCGTTGGTACTGCGCGGATATGGTGCGGGCAATGACGTTACAGCTG...,1,True
3,[thrC],[b0004],[3733:5020](+),[3633:3783](+),forward,CGTTGCCGACTGGTTGGGTAAGAACTACCTGCAAAATCAGGAAGGT...,1,True
4,[yaaX],[b0005],[5233:5530](+),[5133:5283](+),forward,GTAACTTAGAGATTAGGATTGCGGAGAATAACAACCGCCGTTCTCA...,1,True
5,[yaaA],[b0006],[5682:6459](-),[6409:6559](-),reverse,GGACGCGTGGGATGATGTTTCGCAGGAGTAATCACAACTATCGATC...,1,False
6,[yaaJ],[b0007],[6528:7959](-),[7909:8059](-),reverse,ACGCAGAAGTTATCAAGTACCTCGTAGCGTATATACTTCTTAAACA...,1,True
7,[talB],[b0008],[8237:9191](+),[8137:8287](+),forward,ATAACCGTCTTGTCGGCGGTTGCGCTGACGTTGCGTCGTGATATCA...,1,False
8,[mog],[b0009],[9305:9893](+),[9205:9355](+),forward,ACCGGGAAGTCGGTCACGCTACCTCTTCTGAAGCCTGTCTGTCACT...,1,False
9,[satP],[b0010],[9927:10494](-),[10444:10594](-),reverse,CACTGCCCCACTGGCTGATTATTATGCCGCGCCCTGAAAACACTAC...,1,True


# Train, Validation, and Test Sets

The test set will contain 830 sequences, 415 promoters and 415 non-promoters.

The validation set will contain 750 sequences, 375 promoters and 375 non-promoters.

In [34]:
test_proms = df[(df.Promoter == 1) & (df.Independent == True)][:415]
val_proms = df[(df.Promoter == 1) & (df.Independent == True)][415:415+375]

test_genes = df[(df.Promoter == 0) & (df.Independent == True)][:415]
val_genes = df[(df.Promoter == 0) & (df.Independent == True)][415:415+375]

In [35]:
val_set = pd.concat([val_genes, val_proms])
test_set = pd.concat([test_genes, test_proms])

In [36]:
test_val = pd.concat([val_set, test_set])
train_data = df[~df.index.isin(test_val.index)]

In [48]:
def save_to_fasta(filename, data):
  with open(Path(path, filename + '.fa'), 'w') as handle:
    for index, seq in data.iterrows():
      SeqIO.write(SeqRecord(Seq(seq.Sequence), str(seq['Locus']) + "_" + seq['Sample Location'], description=""), handle, 'fasta')

In [49]:
train_data_promoter = train_data[train_data.Promoter == 1]
save_to_fasta('positive_train', train_data_promoter)
train_data_non_promoter = train_data[train_data.Promoter == 0]
save_to_fasta('negative_train', train_data_promoter)

In [52]:
valid_data_promoter = val_set[val_set.Promoter == 1]
save_to_fasta('positive_valid', valid_data_promoter)
valid_data_non_promoter = val_set[val_set.Promoter == 0]
save_to_fasta('negative_valid', valid_data_non_promoter)

In [53]:
test_data_promoter = test_set[test_set.Promoter == 1]
save_to_fasta('positive_test', test_data_promoter)
test_data_non_promoter = test_set[test_set.Promoter == 0]
save_to_fasta('negative_test', test_data_non_promoter)